In [1]:
import numpy as np
import pandas as pd
from time import time
from collections import deque
from functools import reduce
from sklearn import tree

from DecisionTreeClassifier import DecisionTreeClassifier

In [2]:
def train_test_split(X, ratio=0.8):
    X_shuffled = np.copy(X)
    np.random.shuffle(X_shuffled)
    return X_shuffled[:int(len(X_shuffled) * ratio)], X_shuffled[int(len(X_shuffled) * ratio):]

In [3]:
df = pd.read_csv('weatherAUS.csv')

In [4]:
df.dropna(inplace=True)

In [5]:
for feature in ['Location', 'WindGustDir', 'WindGustSpeed', 'WindDir9am', 'WindDir3pm', 'RainToday', 'RainTomorrow']:
    df[feature] = df[feature].astype('category')
    df[feature] = df[feature].cat.codes

In [6]:
df.corr()

,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,...,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RISK_MM,RainTomorrow
Location,1.000000,-0.155954,-0.147500,-0.043314,-0.011509,-0.033941,0.099918,0.085564,-0.033247,0.085732,...,0.026147,0.088567,0.120987,0.018257,0.010527,-0.194449,-0.148955,-0.006037,-0.032842,-0.005100
MinTemp,-0.155954,1.000000,0.749687,0.110800,0.506764,0.076087,-0.190554,0.118389,-0.058310,-0.194123,...,0.070827,-0.475661,-0.495577,0.106177,0.042173,0.906295,0.727222,0.048953,0.142357,0.087428
MaxTemp,-0.147500,0.749687,1.000000,-0.069981,0.650964,0.463913,-0.244508,0.035874,-0.246043,-0.200750,...,-0.448116,-0.350980,-0.451691,-0.263634,-0.257752,0.893387,0.984841,-0.221810,-0.016697,-0.147467
Rainfall,-0.043314,0.110800,-0.069981,1.000000,-0.077239,-0.246379,0.067350,0.106213,0.098980,0.059229,...,0.277625,-0.180606,-0.137302,0.217169,0.191433,0.013713,-0.074627,0.550515,0.304971,0.254342
Evaporation,-0.011509,0.506764,0.650964,-0.077239,1.000000,0.369250,-0.104656,0.209869,-0.130683,-0.072357,...,-0.422436,-0.297319,-0.325954,-0.199809,-0.202366,0.593122,0.630727,-0.218079,-0.044199,-0.130002
Sunshine,-0.033941,0.076087,0.463913,-0.246379,0.369250,1.000000,-0.087355,-0.052108,-0.108689,-0.050795,...,-0.629299,0.043330,-0.020632,-0.677939,-0.702022,0.289038,0.486129,-0.328904,-0.299330,-0.453407
WindGustDir,0.099918,-0.190554,-0.244508,0.067350,-0.104656,-0.087355,1.000000,0.119771,0.430859,0.619892,...,0.053719,-0.118875,-0.022979,0.078431,0.079285,-0.233490,-0.258929,0.148775,-0.003497,0.061751
WindGustSpeed,0.085564,0.118389,0.035874,0.106213,0.209869,-0.052108,0.119771,1.000000,0.023495,0.113683,...,-0.042797,-0.430544,-0.383901,0.088075,0.131439,0.085912,-0.000060,0.146271,0.138414,0.233167
WindDir9am,-0.033247,-0.058310,-0.246043,0.098980,-0.130683,-0.108689,0.430859,0.023495,1.000000,0.320818,...,0.189645,-0.034289,0.060506,0.115243,0.069308,-0.152185,-0.252951,0.181461,-0.000715,0.035992
WindDir3pm,0.085732,-0.194123,-0.200750,0.059229,-0.072357,-0.050795,0.619892,0.113683,0.320818,1.000000,...,-0.004667,-0.120668,-0.021374,0.049014,0.057746,-0.212512,-0.214044,0.128209,-0.021684,0.032203


In [7]:
X = df[['WindGustSpeed', 'RainToday', 'Sunshine', 'Humidity3pm']]
y = df['RainTomorrow']

In [8]:
X_train, X_test = train_test_split(X, ratio=0.9)
y_train, y_test = train_test_split(y, ratio=0.9)

In [13]:
model = DecisionTreeClassifier(min_members=30, criterion='gini')

In [14]:
a=time()
model.fit(X_train, y_train)
b=time()

print('duration: ', b-a)

duration:  0.0343317985534668


In [15]:
a=time()
accuracy = model.better_score(X_test, y_test)
b=time()
print('accuracy: ', accuracy)
print('duration: ', b-a)

accuracy:  0.771357674583481
duration:  0.0004837512969970703


In [16]:
a=time()
accuracy = model.score(X_test, y_test)
b=time()
print('accuracy: ', accuracy)
print('duration: ', b-a)

accuracy:  0.771357674583481
duration:  0.0014033317565917969


In [13]:
"""
a=time()
accuracy = model.score(X_train, y_train)
b=time()


print('time:', b-a)
print('accuracy: ', accuracy)
"""


"\na=time()\naccuracy = model.score(X_train, y_train)\nb=time()\n\n\nprint('time:', b-a)\nprint('accuracy: ', accuracy)\n"

In [14]:
"""
a=time()
accuracy = model.score(X_test, y_test)
b=time()


print('time:', b-a)
print('accuracy: ', accuracy)
"""

"\na=time()\naccuracy = model.score(X_test, y_test)\nb=time()\n\n\nprint('time:', b-a)\nprint('accuracy: ', accuracy)\n"

In [32]:
model_s = tree.DecisionTreeClassifier(min_samples_leaf=30, criterion='entropy')

In [33]:
model_s.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=30, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [34]:
model_s.score(X_test, y_test)

0.7908543069833393